In [2]:
import sys
import json
import numpy as np
params = {}
default = {'training_folder': '../../scenarios/houston/', 'params': params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../../scenarios/houston/', 'params': {}}


In [3]:
def set_index(df):
    if 'index' in df.columns:
        df = df.set_index('index')
    else:
        df.index.name='index'
    return df

In [4]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
num_cores = nb.config.NUMBA_NUM_THREADS
print('num cores:',num_cores)

io_engine= 'pyogrio' if on_lambda else 'pyogrio' #or fiona

PyTables is not installed. No support for HDF output.
num cores: 8


In [5]:
base_folder = argv['training_folder']
input_folder = os.path.join(base_folder,'inputs/')
pt_folder  = os.path.join(input_folder,'pt/')
road_folder = os.path.join(input_folder,'road/')
od_folder =  os.path.join(input_folder,'od/')
params_folder = input_folder if on_lambda else '../../inputs/'

output_folder = os.path.join(base_folder,'outputs/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
model_folder = os.path.join(input_folder, 'model/')

In [6]:
var = excel.read_var(file=os.path.join(params_folder,'parameters.xlsx'), scenario='base', return_ancestry=False)
if 'params' in argv.keys():
    var.update(pd.DataFrame.from_dict(argv['params'], orient="index").stack())

# inputs

In [7]:
sm = stepmodel.read_zippedpickles(model_folder + 'distribution')

zones: 100%|██████████| 28/28 [00:03<00:00,  9.13it/s]           


# pathfinder

In [8]:

sm.step_pt_pathfinder(
    broken_routes=False,
    broken_modes=True,
    keep_pathfinder=True,
    mode_column='route_type',
    route_column='route_id',
    speedup=True,
    walk_on_road=True,
    path_analysis=False,
    od_set= None,
    engine='b', # b is faster!
)


start publicpathfinder
build_mode_combinations
find_broken_mode_paths


breaking modes: set() : 100%|██████████| 4/4 [00:57<00:00, 14.35s/it]          


path_analysis


In [9]:

sm.step_road_pathfinder(method='aon', access_time='time',path_analysis=True,)

In [10]:
sm.analysis_pt_los(walk_on_road=True)
sm.analysis_pt_route_type(hierarchy=[ 'car','rail', 'subway', 'tram', 'bus', 'walk'])

path_analysis: 100%|██████████| 611438/611438 [00:52<00:00, 11720.30it/s]


In [11]:
sm.pt_los['route_type'].unique()

array(['walk', 'tram', 'bus'], dtype=object)

In [12]:
pd.set_option('display.max_columns', None)
sm.links

,trip_id,route_id,agency_id,direction_id,a,b,shape_dist_traveled,link_sequence,time,headway,pickup_type,drop_off_type,route_short_name,route_type,route_color,length,speed,route_width,geometry,disaggregated_a,disaggregated_b,duration,cost,origin,destination
index,,,,,,,,,,,,,,,,,,,,,,,,,
link_163,HOU_002_0,HOU_002,HOU,0,577,577,0.6128,1,74,895,0,0,002,bus,004080,53,2.582671,3,"LINESTRING (245768.905 3289173.777, 245782.451...",node_12987,node_7974,74,521.5,577,577
link_164,HOU_002_0,HOU_002,HOU,0,577,173,0.2991,2,36,895,0,0,002,bus,004080,253,25.093152,3,"LINESTRING (245782.451 3289122.224, 245792.715...",node_7974,node_6317,36,483.5,577,173
link_165,HOU_002_0,HOU_002,HOU,0,173,173,0.3972,3,48,895,0,0,002,bus,004080,20,1.535987,3,"LINESTRING (245789.620 3288871.159, 245790.224...",node_6317,node_6171,48,495.5,173,173
link_166,HOU_002_0,HOU_002,HOU,0,173,173,0.2793,4,33,895,0,0,002,bus,004080,153,16.589633,3,"LINESTRING (245790.224 3288850.847, 245943.601...",node_6171,node_6172,33,480.5,173,173
link_167,HOU_002_0,HOU_002,HOU,0,173,3854,0.3332,5,40,895,0,0,002,bus,004080,462,41.639253,3,"LINESTRING (245943.601 3288841.925, 246405.570...",node_6172,node_6173,40,487.5,173,3854
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
link_12567,HOU_900_1,HOU_900,HOU,1,1392,1109,1.2178,5,229,716,0,0,900,tram,40007E,201,3.157925,5,"LINESTRING (273625.975 3293159.391, 273788.384...",node_8731,node_8732,229,587.0,1392,1109
link_12568,HOU_900_1,HOU_900,HOU,1,1109,1109,0.7219,6,114,716,0,0,900,tram,40007E,105,3.313609,5,"LINESTRING (273788.384 3293041.320, 273875.738...",node_8732,node_8734,114,472.0,1109,1109
link_12569,HOU_900_1,HOU_900,HOU,1,1109,2199,1.4031,7,171,716,0,0,900,tram,40007E,588,12.340743,5,"LINESTRING (273875.738 3292982.438, 273983.709...",node_8734,node_8737,171,529.0,1109,2199


In [13]:
sm.to_zippedpickles(model_folder +'los', omitted_attributes=['publicpathfinder'])

publicpathfinder: 100%|██████████| 29/29 [01:20<00:00,  2.76s/it]
